# 激光点云配准实验
在自动驾驶中，常常需要将不同激光雷达或不同时刻的激光点云进行配准，这样才能进行后续的处理。

本实验使用python-pcl来进行激光点云配准

## 1. 环境配置
本实验使用python-pcl来进行。python-pcl是PCL点云库的python封装，实现了PCL的部分功能。Point Cloud Lirary(PCL）是用于进行点云处理任务和3D几何处理任务的开源库，该库包含用于特征估计，表面重建，3D配准，模型拟合和分割的算法。PCL是用C++编写的，并在BSD许可下发布。

python-pcl对PCL的基础部分进行了封装，方便python开发者进行点云处理。

### 1.1 python-pcl安装和配置
python-pcl在linux、Max OS和Windows上的安装可参考官方github页面:https://github.com/strawlab/python-pcl 。此外，由于python-pcl安装较为复杂，因此本实验推荐使用python-pcl的docker镜像来进行实验。python-pcl的docker镜像可从官方```docker/Dockerfile```文件构建，也可在docker-hub中去寻找。

### 1.2 CloudCompare安装
CloudCompare是3D点云处理软件（例如使用激光扫描仪获得的软件），可以处理点云、三角形网格和校准的图像等数据。
CloudCompare项目最初是由巴黎电信技术公司与EDF的研发部门合作创建的，始于2003年，由Daniel Girardeau-Montaut博士研究3D几何数据的变化。当时，其主要目的是快速检测在工业设施（例如发电厂）或建筑工地中使用激光扫描仪采集的3D高密度点云的变化。之后，它演变为更通用，更高级的3D数据处理软件。 现在，它是一个独立的开源项目和一个免费软件。

本实验将使用CloudCompare来对比配准之前的点云与配准之后的点云。ClourCompare可在Linux、Mac OS和Windows中运行，其下载和安装在官方网站(https://www.danielgm.net/cc/ )中。

![cloudcompare](cloudcompare.jpg)

## 2. 激光点云配准实验
通常, 同一条道路上不同批次采集的激光 点 云 数 据 存 在 误 差。这 些 误 差 体 现 在 同 一 物
体激光点云的 x、
 y、
 z 坐标在不同批次采集的激光点云数据中不完全相等。接下来, 以地面
激光点云的 z 轴坐标为例进行说明。假设以激光雷达为中心坐标, 通 过 激 光 射 线 发 送 与 反
射回来的时间计算中心点与地 面 的 距 离, 然 后 将 这 个 距 离 转 换 为 地 面 某 一 点 的 z 轴 坐 标,
如果采集车两次采集的激光雷达距离地面高度不同, 射线经历的时间也不同, 最后计算出的
距离就会不同, 从而造成两次 采 集 的 z 轴 坐 标 有 偏 差。 当 然, 造 成 两 组 数 据 坐 标 不 等 并 不
止这一种情况, 因此, 需要激光点云配准, 将不 同 批 次 采 集 的 激 光 点 云 数 据 融 合 在 一 起。接
下来介绍激光点云配准实验。


In [1]:
import numpy as np
import pcl

### 2.1 读取数据
首先我们先读取待匹配的激光点云数据。激光点云数据在```data/```文件夹下，以pcd(binary)的格式存储。

In [2]:
# 读取点云
def txt_reader(fileName):
    """
    :param fileName: 点云文件，格式为pcb
    :return: 返回点云
    """
    cloud_ = pcl.load(fileName)

    return cloud_

sour_cloud = txt_reader("data/128.pcd")
dist_cloud = txt_reader("data/288.pcd")

### 2.2 KdTree搜索最近邻
根据匹配思路，我们需要得到sour_cloud中每个点云在dist_cloud中的近邻点，计算出待匹配点云的平均误差来进行匹配。

直观来说，我们可以通过遍历两片点云中所有点的方式来搜索最近邻，但这种搜索方法时间复杂度为$O(n^2$，而激光点云的数据量是非常大，如上述的```data/128.pcd```中有90多万个点，因此这种低效的搜索方式不适合我们使用。为了提高搜索效率，我们使用KdTree的方式来搜索最近邻。

使用KdTree搜索最近邻的函数如下所示：

In [3]:
# 使用KdTree搜索两片点云的最近邻
def Kdsearch(sour_pole, dist_pole):
    """
    使用KDTree搜索最近
    :param sour_pole: Kdtree搜索的源点云
    :param dist_pole: Kdtree搜索的搜索点云
    :return: 误差均值向量
    """
    cloud_ = pcl.PointCloud()

    size_of_sourPole = sour_pole.width
    size_of_distPole = dist_pole.width


    points = np.zeros((size_of_sourPole, 3), dtype = np.float32)
    for i in range(0, size_of_distPole):
        points[i][0] = dist_pole[i][0]
        points[i][1] = dist_pole[i][1]
        points[i][2] = dist_pole[i][2]
    cloud_.from_array(points)

    radius = 1.0
    match_size = 0

    matched_flags = np.zeros((size_of_sourPole, 1))
    kdtree = cloud_.make_kdtree_flann()
    allMeanVector = np.zeros((1, 3), dtype = np.float32)
    # 为了方便去前100
    for i in range(0, size_of_sourPole):
        # 设置待搜索的点
        searchPointPCL = pcl.PointCloud()
        searchPoint = np.zeros((1, 3), dtype = np.float32)
        searchPoint[0][0] = sour_cloud[i][0]
        searchPoint[0][1] = sour_cloud[i][1]
        searchPoint[0][2] = sour_cloud[i][2]
        searchPointPCL.from_array(searchPoint)
        # print("搜索点为 x: " + str(searchPointPCL[0][0]) + ', y : ' +
        #       str(searchPointPCL[0][1]) + ', z : ' + str(searchPointPCL[0][2]))

        # 使用kdtree搜索K近邻
        [ind, sqdist] = kdtree.nearest_k_search_for_cloud(searchPointPCL, 10)
        # [ind, sqdist] = kdtree.radius_search_for_cloud(searchPointPCL, radius)
        errVector = np.zeros((1, 3), dtype = np.float32)
        for k in range(0, ind.size):
            if(matched_flags[ind[0][k]][0] == 0):
                # print('匹配点为: (' + str(cloud_[ind[0][k]][0]) + ' ' + str(cloud_[ind[0][k]][1]) + ' ' + str(
                #     cloud_[ind[0][k]][2]) + ' (squared distance: ' + str(sqdist[0][k]) + ')')
                # 计算当前的平均误差
                if(sqdist[0][k] < radius):
                    errVector[0][0] = errVector[0][0] + searchPointPCL[0][0] - cloud_[ind[0][k]][0]
                    errVector[0][1] = errVector[0][1] + searchPointPCL[0][1] - cloud_[ind[0][k]][1]
                    errVector[0][2] = errVector[0][2] + searchPointPCL[0][2] - cloud_[ind[0][k]][2]

                    # 将当前flag设为1
                    matched_flags[ind[0][k]][0] = 1
                    match_size += 1

        allMeanVector += errVector

    print("matched size: ", match_size)
    print("mean errVector: ", allMeanVector/match_size)

    return allMeanVector/match_size

### 2.3 点云匹配
通过上述函数，我们可以找到待匹配点云的平均误差。接下来，根据这个平均误差，我们可以对点云进行匹配：

In [4]:
# 匹配两片点云
def mix(sour_pole, dist_pole, save_file_name):
    """
    混合两片点云
    :param sour_pole: 第一片点云
    :param dist_pole: 第二片点云
    :return: None
    """
    d_value_vector = Kdsearch(sour_pole, dist_pole)

    # 创建新的点云
    p_sour = pcl.PointCloud()
    p_sour_data = np.zeros((sour_pole.width, 3), dtype = np.float32)

    for i in range(0, sour_pole.width):
        # print(sour_pole[i][0])
        p_sour_data[i][0] = sour_pole[i][0] - d_value_vector[0][0]
        p_sour_data[i][1] = sour_pole[i][1] - d_value_vector[0][1]
        p_sour_data[i][2] = sour_pole[i][2] - d_value_vector[0][2]
    p_sour.from_array(p_sour_data)


    pcl.save(p_sour, save_file_name, 'pcd')
    print("successful save new pcd file")

In [5]:
mix(sour_cloud, dist_cloud, "data/288_1.pcd")

('matched size: ', 212909)
('mean errVector: ', array([[-0.00842201,  0.00449172,  0.07825302]]))
successful save new pcd file


## 3. 实验结果
匹配之后的点云保存为```data/288_1.pcd```，我们可以通过CloudCompare来对比配准前后的点云。

配准之前的点云：

![before](BeforeMix.jpg)

如图所示，匹配之前的点云明显存在着误差。

匹配之后的点云：

![after](AfterMinx.jpg)

匹配之后，点云之间的差异被大大减小了